> __CHANGE IF NEED BE__

In [1]:
data_source = "/home/boturon/proj/git/parlamento/data/"
parliament_count = ["02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13"]

________

__ Dependencies __

In [2]:
import pandas as pd
import json

<br>

# Biographic Information

__ Variables __

In [9]:
col_names_label_bio = ["id", "name", "gender", "profession_law", "legislatures_count", "legislatures_last", "party_last", "district_last"]

__ Functions __

In [11]:
def table_bio(source):
    # create DataFrame for parlamentarians
    df_bio = pd.DataFrame(index = col_names_label_bio)
    
    # start index
    row = 0
    
    for count in parliament_count:
        # open file
        data_parlamentarians = json.loads(open(source+"registo-biografico-"+count+".json").read())
        
        for parlamentarian in data_parlamentarians["RegistoBiografico"]["RegistoBiograficoList"]["pt_ar_wsgode_objectos_DadosRegistoBiograficoWeb"]:
            # index
            row += 1
            
            # get data for each label
            parlamentarian_id = parlamentarian["cadId"]
            parlamentarian_name = parlamentarian["cadNomeCompleto"]
            parlamentarian_gender = parlamentarian["cadSexo"]
            
            # dummy for law related jobs
            if  parlamentarian.get("cadProfissao", "None").lower() in ["jurista", "advogado", "advogada"]:
                parlamentarian_profession_law = 1
            else:
                parlamentarian_profession_law = 0
            
            legislatures = parlamentarian["cadDeputadoLegis"]["pt_ar_wsgode_objectos_DadosDeputadoLegis"]
            
            # deal with one-time parlamentarians
            if type(legislatures) == dict:
                legislatures_count = 1
                legislatures_last = legislatures["legDes"]
                party_last = legislatures["gpSigla"]
                district_last = legislatures["ceDes"]
            else:
                legislatures_count = len(legislatures)
                legislatures_last = legislatures[0]["legDes"]
                party_last = legislatures[0]["gpSigla"]
                district_last = legislatures[0]["ceDes"]
                
            # add parlamentarian information to table
            df_bio[row] = [parlamentarian_id, parlamentarian_name, parlamentarian_gender, parlamentarian_profession_law, legislatures_count, legislatures_last, party_last, district_last]
    
    return(df_bio.transpose())

__ Output __

In [12]:
table_bio(data_source).tail()

,id,name,gender,profession_law,legislatures_count,legislatures_last,party_last,district_last
4362,1743,ULISSES MANUEL BRANDÃO PEREIRA,M,0,3,XIII,PSD,AVEIRO
4363,4402,VÂNIA CARVALHO DIAS DA SILVA DE ANTAS DE BARROS,F,0,1,XIII,CDS-PP,BRAGA
4364,1867,JOSÉ ANTÓNIO FONSECA VIEIRA DA SILVA,M,0,4,XIII,PS,SANTARÉM
4365,1700,VITALINO JOSÉ FERREIRA PROVA CANAS,M,0,5,XIII,PS,LISBOA
4366,466,WANDA OLAVO CORREA D'AZEVEDO GUIMARÃES,F,0,1,XIII,PS,LISBOA


________

# Initiatives

__ Variables __

In [57]:
col_names_label_init = ["init_id", "init_year_end", "init_type"]#, "authors_count"]

__ Functions __

In [64]:
def table_init(source):
    # create DataFrame for parlamentarians
    df_init = pd.DataFrame(index = col_names_label_init)
    
    # start index
    row = 0
    
    for count in parliament_count:
        # open file
        data_initiatives = json.loads(open(source+"iniciativas-"+count+".json").read())
            
        for initiative in data_initiatives["ArrayOfPt_gov_ar_objectos_iniciativas_DetalhePesquisaIniciativasOut"]["pt_gov_ar_objectos_iniciativas_DetalhePesquisaIniciativasOut"]:
                              
            # get data for each label
            init_id = initiative["iniId"]
            init_type = initiative["iniDescTipo"]
            
            #authors = initiative["iniAutorDeputados"]["pt_gov_ar_objectos_iniciativas_AutoresDeputadosOut"]
            #print(authors)
            #if type(authors) == dict:
            #    authors_count = 1
            #else:
            #    authors_count = len(authors)
                
            # only add if initiative is finished
            try:
                # index
                row += 1
                # Final year
                init_year_end = int(initiative["dataFimleg"][0:4])
                # add parlamentarian information to table
                df_init[row] = [init_id, init_year_end, init_type]#, authors_count]
            except KeyError:
                continue
                        
    return(df_init.transpose())

__ Output __

In [65]:
food_for_ze = table_init(data_source).tail()

In [66]:
food_for_ze.to_csv('./food_for_ze.csv')